## Uvod

Prikazana je uporaba PostgreSQL razširitve **pgvector** za delo z vektorskimi podatki. Namen je prikazati celoten postopek obdelave in analize podatkov na podlagi embeddingov, od začetne priprave podatkov do izvedbe naprednih poizvedb. Naloga je zasnovana okoli podatkov o izdelkih, uvoženih z Amazona, kjer vsak izdelek vsebuje atribute, kot so kategorija, naslov, opis, lastnosti in druge podrobnosti.

### Vsebina naloge:
1. **Priprava podatkov:**
   - Uvoz podatkov o izdelkih iz Amazona v obliki JSONL datoteke.
   - Podatki prebrani, prečiščeni in uvoženi v PostgreSQL tabelo z ustrezno strukturo.

2. **Generiranje embeddingov:**
   - Z uporabo modela **SentenceTransformer** so za vsak izdelek generirani embeddingi, ki predstavljajo vektorsko predstavitev atributov izdelka (naslov, opis, lastnosti, kategorije).
   - Generirani embeddingi so shranjeni v PostgreSQL tabeli.

3. **CRUD operacije:**
   - Osnovne operacije ustvarjanja, branja, posodabljanja in brisanja zapisov v tabeli.
   - Meritev časa izvajanja in porabo pomnilnika.

4. **Poizvedbe brez indeksa:**
   - Izvedba poizvedb za iskanje podobnih izdelkov na podlagi embeddingov, brez uporabe indeksov.

5. **Uporaba indeksa:**
   - Na stolpcu embedding uporaba **ivfflat indeksa**, ki omogoča optimizacijo iskanja najbližjih sosedov.
   - Ponovitev ključnih poizvedb za primerjavo zmogljivost brez in z indeksom.

6. **Analiza rezultatov:**
   - Primerjava časa izvajanja in poraba pomnilnika med poizvedbami brez indeksa in z indeksom.


In [ ]:
import gzip
import json

# Pot do datoeke
file_path = r'C:\Users\sara\Downloads\meta_Musical_Instruments.jsonl\meta_Musical_Instruments.jsonl'

# Razpakiramo datoteko in preberemo vsebino
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Izpis prvih nekaj zapisov
for i, record in enumerate(data[:5]):  # Prikažemo prvih 5 zapisov
    print(f"Zapis {i + 1}: {record}")


In [ ]:
# Izpis prvega elementa
first_record = data[0]

# Prikaz strukture prvega elementa
print("Prvi zapis:")
for key, value in first_record.items():
    print(f"{key}: {value}")

Prvi zapis:  
main_category: Musical Instruments  
title: Pearl Export Lacquer EXL725S/C249 5-Piece New Fusion Drum Set with Hardware, Honey Amber  
average_rating: 4.2  
rating_number: 22  
features: ['Item may ship in more than one box and may arrive separately', '(22x18, 10x7, 12x8, 16x16, 14x5.5)', 'P930 Demonator Pedal', '830 Hardware Pack', 'Matching snare, REMO snare batter side head']  
description: ["Introducing the best selling drum set of all time... Export Series returns and this time with a lacquer finish. EXL Export Lacquer Series incorporates Pearl's S.S.T. Superior Shell Technology, Opti-Loc tom mounts, all-new 830 Series Hardware with a P-930 Pedal, and a choice of three amazing stocking finishes."]  
price: None  
images: [{'thumb': 'https://m.media-amazon.com/images/I/51QYyZgky-L._AC_US40_.jpg', 'large': 'https://m.media-amazon.com/images/I/51QYyZgky-L._AC_.jpg'}]  
videos: [{'title': 'Best Selling Drum Set of All Time', 'url': 'https://www.amazon.com/vdp/0be29e27175a4280aa9c2027ec3664b4?ref=dp_vse_rvc_0', 'user_id': ''}]  
store: Pearl  
categories: ['Musical Instruments', 'Drums & Percussion', 'Drum Sets & Set Components', 'Drum Sets']  
details: {'Item Weight': '33 pounds', 'Product Dimensions': '22 x 22 x 20 inches', 'Item model number': 'EXL725S/C249', 'Best Sellers Rank': {'Musical Instruments': 150300, 'Drum Sets': 521}, 'Date First Available': 'April 4, 2014', 'Color Name': 'Honey Amber', 'Material Type': 'Pearl', 'Size': '-inch', 'Color': 'Honey Amber', 'Brand': 'Pearl', 'Material': 'Pearl', 'Model Name': 'EXL725S', 'Item Dimensions LxWxH': '22 x 22 x 20 inches'}  
parent_asin: B01M4HO6RK  
bought_together: None  

In [ ]:
# Omejimo podatke na pomembne atribute
filtered_data = []
for record in data[:10000]:  # Omejimo na 10,000 zapisov
    filtered_record = {
        "main_category": record.get("main_category", None),  #Ce ne najde categorije postavi na None
        "title": record.get("title", None),
        "description": record.get("description", None),
        "features": record.get("features", []),
        "categories": record.get("categories", [])
    }
    filtered_data.append(filtered_record)


In [ ]:
# Izpis prvega elementa
first_record = filtered_data[0]

# Prikaz strukture prvega elementa
print("Prvi zapis:")
for key, value in first_record.items():
    print(f"{key}: {value}")

Prvi zapis:  
main_category: Musical Instruments  
title: Pearl Export Lacquer EXL725S/C249 5-Piece New Fusion Drum Set with Hardware, Honey Amber  
description: ["Introducing the best selling drum set of all time... Export Series returns and this time with a lacquer finish. EXL Export Lacquer Series incorporates Pearl's S.S.T. Superior Shell Technology, Opti-Loc tom mounts, all-new 830 Series Hardware with a P-930 Pedal, and a choice of three amazing stocking finishes."]  
features: ['Item may ship in more than one box and may arrive separately', '(22x18, 10x7, 12x8, 16x16, 14x5.5)', 'P930 Demonator Pedal', '830 Hardware Pack', 'Matching snare, REMO snare batter side head']  
categories: ['Musical Instruments', 'Drums & Percussion', 'Drum Sets & Set Components', 'Drum Sets']  

In [1]:
import psycopg2

# Povezava do PostgreSQL baze
try:
    conn = psycopg2.connect(
        host="localhost", 
        port="5433",  
        database="product_db",  
        user="postgres",
        password="password"  
    )
    print("Povezava do baze je bila uspešno vzpostavljena!")
except Exception as e:
    print("Napaka pri povezavi:", e)
    conn = None

Povezava do baze je bila uspešno vzpostavljena!


In [10]:
cur = conn.cursor()
# Ustvari tabelo v PostgreSQL
create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    id SERIAL PRIMARY KEY,
    main_category TEXT,
    title TEXT,
    description TEXT,
    features TEXT[],      
    categories TEXT[]     
);
"""
cur.execute(create_table_query)
conn.commit()
print("Tabela 'products' je bila uspešno ustvarjena!")

Tabela 'products_filtered' je bila uspešno ustvarjena!


In [11]:
cur = conn.cursor()
# SQL za vstavljanje podatkov
insert_query = """
INSERT INTO products (main_category, title, description, features, categories)
VALUES (%s, %s, %s, %s, %s);
"""

# Vstavljanje podatkov
for record in filtered_data:
    cur.execute(insert_query, (
        record["main_category"],
        record["title"],
        record["description"],
        record["features"],
        record["categories"]
    ))
conn.commit()
print("10,000 zapisov je bilo uspešno vstavljenih v tabelo 'products'!")

10,000 zapisov je bilo uspešno vstavljenih v tabelo 'products'!


In [2]:
cur = conn.cursor()
# Preveri število zapisov
cur.execute("SELECT COUNT(*) FROM products;")
count = cur.fetchone()
conn.commit()
print("Število zapisov v tabeli 'products':", count[0])

# Pridobi prvih nekaj zapisov
cur.execute("SELECT * FROM products LIMIT 1;")
rows = cur.fetchall()
conn.commit()
for row in rows:
    print(row)

Število zapisov v tabeli 'products': 10000
(1, 'Musical Instruments', 'Pearl Export Lacquer EXL725S/C249 5-Piece New Fusion Drum Set with Hardware, Honey Amber', '{"Introducing the best selling drum set of all time... Export Series returns and this time with a lacquer finish. EXL Export Lacquer Series incorporates Pearl\'s S.S.T. Superior Shell Technology, Opti-Loc tom mounts, all-new 830 Series Hardware with a P-930 Pedal, and a choice of three amazing stocking finishes."}', ['Item may ship in more than one box and may arrive separately', '(22x18, 10x7, 12x8, 16x16, 14x5.5)', 'P930 Demonator Pedal', '830 Hardware Pack', 'Matching snare, REMO snare batter side head'], ['Musical Instruments', 'Drums & Percussion', 'Drum Sets & Set Components', 'Drum Sets'], None)


In [5]:
# Dodajanje stolpca embedding
cur.execute("""
    ALTER TABLE products
    ADD COLUMN embedding vector(384);
""")
conn.commit()

print("Stolpec 'embedding' je bil uspešno dodan!")

Stolpec 'embedding' je bil uspešno dodan!


In [3]:
cur.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'products';
""")
columns = cur.fetchall()
conn.commit()
for column in columns:
    print(column)

('id', 'integer')
('embedding', 'USER-DEFINED')
('title', 'text')
('features', 'ARRAY')
('categories', 'ARRAY')
('description', 'text')
('main_category', 'text')


### GENERIRANJE EMBEDDINGOV

In [ ]:
from sentence_transformers import SentenceTransformer
import time

# Inicializacija modela
model = SentenceTransformer('all-MiniLM-L6-v2')

batch_size = 100  # Velikost sklopa za hitrejše obdelovanje
total_records = len(filtered_data)  # Skupno število zapisov
start_time = time.time()

# Obdelaj zapise v batchih
for i in range(0, total_records, batch_size):
    batch = filtered_data[i:i + batch_size]
    texts = [
        f"{record[1] or ''} {record[2] or ''} {' '.join(record[3] or [])} {' '.join(record[4] or [])}"
        for record in batch
    ]
    
    # Generiraj embeddinge za trenutni batch
    batch_embeddings = model.encode(texts)
    
    # Shrani ID-je in embeddinge v bazo
    for idx, embedding in enumerate(batch_embeddings):
        record_id = batch[idx][0]
        cur.execute("""
            UPDATE products
            SET embedding = %s
            WHERE id = %s;
        """, (embedding.tolist(), record_id))  # Shranimo embedding kot seznam
    conn.commit()  # Potrdimo spremembe po vsakem batchu

    # Izpiši napredek
    elapsed_time = time.time() - start_time
    print(f"Obdelanih {i + len(batch)}/{total_records} zapisov. Čas od začetka: {elapsed_time:.2f} s")

print("Embeddingi uspešno generirani in shranjeni za vse zapise!")


### Rezultati
Obdelanih 100/10000 zapisov. Čas od začetka: 8.13 s  
Obdelanih 200/10000 zapisov. Čas od začetka: 16.19 s  
Obdelanih 300/10000 zapisov. Čas od začetka: 24.51 s  
Obdelanih 400/10000 zapisov. Čas od začetka: 32.87 s  
...  
Obdelanih 9700/10000 zapisov. Čas od začetka: 809.45 s  
Obdelanih 9800/10000 zapisov. Čas od začetka: 817.84 s  
Obdelanih 9900/10000 zapisov. Čas od začetka: 826.27 s  
Obdelanih 10000/10000 zapisov. Čas od začetka: 834.66 s  
Embeddingi uspešno generirani in shranjeni za vse zapise!  

In [7]:
cur.execute("SELECT COUNT(*) FROM products WHERE embedding IS NOT NULL;")
count = cur.fetchone()[0]
print(f"Število izdelkov z generiranimi embeddingi: {count}")

Število izdelkov z generiranimi embeddingi: 10000


# PRIMER UPORABE PGVECTOR - Amazon Products

## Osnovne CRUD Operacije

V tej sekciji bomo prikazali osnovne operacije za delo z bazo podatkov PostgreSQL, kjer je shranjena tabela **products**. Operacije vključujejo:

1. **Create (Ustvarjanje):** Dodajanje novega zapisa v tabelo.
2. **Read (Branje):** Pridobivanje podatkov iz tabele.
3. **Update (Posodabljanje):** Spreminjanje obstoječih podatkov.
4. **Delete (Brisanje):** Odstranjevanje zapisa iz tabele.

Poleg same izvedbe operacij bomo za vsako izmerili čas izvajanja, da dobimo vpogled v učinkovitost.


### CREATE OPERACIJA

In [ ]:
import psutil
import time
from sentence_transformers import SentenceTransformer

# Inicializacija modela
model = SentenceTransformer('all-MiniLM-L6-v2')

# Priprava podatkov
title = "Yamaha Acoustic Guitar"
description = "A high-quality acoustic guitar for beginners and professionals alike."
features = ["6 strings", "Spruce top", "Mahogany back and sides"]
categories = ["Guitars", "Acoustic Guitars"]
text = f"{title} {description} {' '.join(features)} {' '.join(categories)}"

# Generiraj embedding (384 dimenzij)
embedding = model.encode(text).tolist()

# Merjenje pomnilnika
process = psutil.Process()
start_memory = process.memory_info().rss  # Začetna poraba pomnilnika
start_time = time.time()  # Začetni čas

# CREATE: Dodajanje zapisa
cur.execute("""
    INSERT INTO products (main_category, title, description, features, categories, embedding)
    VALUES (%s, %s, %s, %s, %s, %s);
""", (
    "Musical Instruments",
    title,
    description,
    features,
    categories,
    embedding
))
conn.commit()

# Merjenje končne porabe pomnilnika in časa
end_time = time.time()
end_memory = process.memory_info().rss  # Končna poraba pomnilnika

# Izpis rezultatov
print(f"Zapis uspešno dodan.")
print(f"Čas izvajanja: {end_time - start_time:.4f} s")
print(f"Poraba pomnilnika: {end_memory - start_memory} bajtov")


Zapis uspešno dodan.  
Čas izvajanja: 0.0753 s  
Poraba pomnilnika: 4096 bajtov  

### READ OPERACIJA

In [ ]:
import time
import psutil
import os

# Inicializacija spremljanja pomnilnika
process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss  # Pomnilnik na začetku v bajtih
start_time = time.time()  # Čas na začetku

# Poizvedba za izdelek s točno določenim naslovom
cur.execute("SELECT id, title, main_category, description FROM products WHERE title = %s;", ("Yamaha Acoustic Guitar",))
result = cur.fetchone()

# Čas in pomnilnik po izvedbi poizvedbe
end_time = time.time()
end_memory = process.memory_info().rss

# Izpis pomembnih rezultatov
if result:
    print(f"Pridobljen zapis:")
    print(f"ID: {result[0]}, Title: {result[1]}")
    print(f"Category: {result[2]}")
    print(f"Description: {result[3][:50]}...")  # Prikaz prvih 50 znakov opisa
else:
    print("Zapis ni bil najden.")

# Izpis časa in pomnilnika
print(f"Čas izvajanja: {end_time - start_time:.4f} s")
print(f"Poraba pomnilnika: {end_memory - start_memory} bajtov")


Pridobljen zapis:  
ID: 10005, Title: Yamaha Acoustic Guitar  
Category: Musical Instruments  
Description: A high-quality acoustic guitar for beginners and p...  
Čas izvajanja: 0.0140 s  
Poraba pomnilnika: 8192 bajtov  

### UPDATE OPERACIJA

In [ ]:
from sentence_transformers import SentenceTransformer

# Inicializacija modela
model = SentenceTransformer('all-MiniLM-L6-v2')

# Priprava novih podatkov
new_description = "Updated description for the Yamaha Acoustic Guitar."
title = "Yamaha Acoustic Guitar"
features = ["6 strings", "Spruce top", "Mahogany back and sides"]
categories = ["Guitars", "Acoustic Guitars"]

# Združi podatke za generiranje novega embeddinga
text = f"{title} {new_description} {' '.join(features)} {' '.join(categories)}"
new_embedding = model.encode(text).tolist()

# Merjenje pomnilnika in časa
process = psutil.Process()
start_memory = process.memory_info().rss
start_time = time.time()

# UPDATE: Posodobitev zapisa z novim opisom in embeddingom
cur.execute("""
    UPDATE products
    SET description = %s, embedding = %s
    WHERE title = %s;
""", (new_description, new_embedding, title))
conn.commit()

# Merjenje končne porabe pomnilnika in časa
end_time = time.time()
end_memory = process.memory_info().rss

# Izpis rezultatov
print(f"Zapis uspešno posodobljen z novim opisom in embeddingom.")
print(f"Čas izvajanja: {end_time - start_time:.4f} s")
print(f"Poraba pomnilnika: {end_memory - start_memory} bajtov")


Zapis uspešno posodobljen z novim opisom in embeddingom.  
Čas izvajanja: 0.0790 s  
Poraba pomnilnika: 4096 bajtov  

### DELETE OPERACIJA

In [ ]:
# Merjenje pomnilnika
process = psutil.Process()
start_memory = process.memory_info().rss
start_time = time.time()

# DELETE: Brisanje zapisa
cur.execute("""
    DELETE FROM products WHERE title = %s;
""", ("Yamaha Acoustic Guitar",))
conn.commit()

# Merjenje končne porabe pomnilnika in časa
end_time = time.time()
end_memory = process.memory_info().rss

# Izpis rezultatov
print(f"Zapis uspešno izbrisan.")
print(f"Čas izvajanja: {end_time - start_time:.4f} s")
print(f"Poraba pomnilnika: {end_memory - start_memory} bajtov")


Zapis uspešno izbrisan.  
Čas izvajanja: 0.0283 s  
Poraba pomnilnika: 12288 bajtov  

# Poizvedbe  z uporabo pgvector

## 1. Pridobitev najbližjih sosedov (nearest neighbors)

Ta poizvedba vrne izdelke, katerih embeddingi so najbližji določenemu ciljnemu embeddingu, brez uporabe indeksa. Najbližje sosednje izdelke določimo z uporabo funkcije **pgvector** za merjenje razdalje.

### Poizvedba
- Izračun razdalje med ciljnim embeddingom in vsemi embeddingi v tabeli `products`.
- Razvrstitev izdelkov po razdalji (najmanjša razdalja je najbolj podobna).
- Omejitev rezultatov na 10 rezultatov.


In [ ]:
import time
import psutil
import os

# Inicializacija spremljanja pomnilnika
process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss  # Pomnilnik na začetku v bajtih

# Preverimo, ali ima generiran embedding pravilno dimenzijo
target_embedding = model.encode("Acoustic guitar for beginners").tolist()

# Preverjanje dimenzije embeddinga
if len(target_embedding) != 384:
    raise ValueError(f"Embedding mora imeti dimenzijo 384, a ima {len(target_embedding)}.")

# Začetek merjenja časa
start_time = time.time()

# SQL poizvedba za pridobitev najbližjih sosedov z evklidsko razdaljo
cur.execute("""
    SELECT id, title, main_category, embedding <-> %s::vector AS distance
    FROM products
    ORDER BY distance ASC
    LIMIT 10;
""", (target_embedding,))  # Embedding predamo kot parameter

# Prikaz rezultatov
results = cur.fetchall()
end_time = time.time()

# Merjenje pomnilnika po izvedbi
end_memory = process.memory_info().rss  # Pomnilnik na koncu v bajtih

# Izpis rezultatov
print(f"Najbližji sosedi (10 rezultatov):")
for row in results:
    print(f"ID: {row[0]}, Title: {row[1]}, Category: {row[2]}, Distance: {row[3]:.4f}")

# Izračun in izpis časa izvajanja
elapsed_time = end_time - start_time
print(f"Čas izvajanja: {elapsed_time:.4f} s")

# Izračun in izpis porabe pomnilnika
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Pretvorimo v MB
print(f"Poraba pomnilnika: {memory_used:.2f} MB")


### Najdeni rezultati poizvedbe
Najbližji sosedi (10 rezultatov):  
ID: 6237, Title: Best Choice Products 38in Beginner All Wood Acoustic Guitar Starter Kit w/Case, Strap, Digital Tuner, Pick, Strings - SoCal Green, Category: Musical Instruments, Distance: 0.7093  
ID: 6953, Title: Ashthorpe 38-inch Beginner Acoustic Guitar Package (Blue), Basic Starter Kit w/Gig Bag, Strings, Strap, Tuner, Pitch Pipe, Picks, Category: Musical Instruments, Distance: 0.7350  
ID: 415, Title: ADM Acoustic Guitar 38 Inch for Kids Beginner Adults Steel Strings Cutaway Wooden Acustica Guitarra Bundle Starter Kit with Free Learning Lessons Card, Gig Bag, Tuner, Strap, Picks, Capo etc, Blue, Category: Musical Instruments, Distance: 0.7601  
ID: 9799, Title: Full Size Dreadnaught Mahogany Acoustic Guitar With Six Steel String Gig Bag For Beginners - Natural, Category: Musical Instruments, Distance: 0.7642  
ID: 5937, Title: Best Choice Products Beginner Acoustic Electric Guitar Starter Set 38in w/All Wood Cutaway Design, Case, Strap, Picks, Tuner - Blue, Category: Musical Instruments, Distance: 0.7753  
ID: 7237, Title: Asmuse 38 inch Acoustic Guitar Kit, Full Size Classical Acoustic Guitar 6 Strings with Gig Bag, Tuner, Picks, Strap Accessories for Beginners Adults Teens (Blue), Category: Musical Instruments, Distance: 0.7764  
ID: 4191, Title: LAGRIMA 38 inch 4/4 Size Beginner Acoustic Guitar Set Starter Kit with Gig Bag, Picks & Steel Strings for Kids/Beginners/Adults (Blue), Category: Musical Instruments, Distance: 0.7924  
ID: 9374, Title: 38" Wood Guitar With Case and Accessories for Kids/Boys/Girls/Teens/Beginners (Pink), Category: Musical Instruments, Distance: 0.8129  
ID: 2721, Title: Guitar Beginner,One-key chord assisted learning tool for guitar beginners Guitar Practice Aid Tool for Adults & Children Trainer Beginners (white), Category: Musical Instruments, Distance: 0.8410  
ID: 343, Title: Beginner Acoustic Guitar Ranch 41" Full Size Solid Wood Cutaway Beginners Steel String Guitars Kit Bundle with Gig Bag/Tuner/Capo/Strings/Strap/Picks Set Starter Guitar Pack for Adults (Dreadnought), Category: Musical Instruments, Distance: 0.8439  
#### BREZ INDEKSA
Čas izvajanja: 0.1224 s  
Poraba pomnilnika: 0.02 MB  
#### Z INDEKSOM  
Čas izvajanja: 0.1081 s  
Poraba pomnilnika: 0.09 MB

## Poizvedba 2: Najbližji produkti znotraj določene kategorije

Ta poizvedba uporablja stolpec `embedding` za iskanje najbližjih produktov določenemu ciljnemu vektorju, ki pripadajo določeni kategoriji. 

In [ ]:
import time
import psutil
import os

# Inicializacija spremljanja pomnilnika
process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss  # Pomnilnik na začetku v bajtih

# Ciljna kategorija in embedding
target_category = "Musical Instruments"
target_embedding = model.encode("Electric guitar for rock music").tolist()

# Preverjanje dimenzije embeddinga
if len(target_embedding) != 384:
    raise ValueError(f"Embedding mora imeti dimenzijo 384, a ima {len(target_embedding)}.")

# Začetek merjenja časa
start_time = time.time()

# SQL poizvedba za pridobitev najbližjih sosedov znotraj kategorije
cur.execute("""
    SELECT id, title, main_category, embedding <-> %s::vector AS distance
    FROM products
    WHERE main_category = %s
    ORDER BY distance ASC
    LIMIT 10;
""", (target_embedding, target_category))

# Prikaz rezultatov
results = cur.fetchall()
end_time = time.time()

# Merjenje pomnilnika po izvedbi
end_memory = process.memory_info().rss  # Pomnilnik na koncu v bajtih

# Izpis rezultatov
print(f"Najbližji sosedi znotraj kategorije '{target_category}' (10 rezultatov):")
for row in results:
    print(f"ID: {row[0]}, Title: {row[1]}, Category: {row[2]}, Distance: {row[3]:.4f}")

# Izračun in izpis časa izvajanja
elapsed_time = end_time - start_time
print(f"Čas izvajanja: {elapsed_time:.4f} s")

# Izračun in izpis porabe pomnilnika
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Pretvorimo v MB
print(f"Poraba pomnilnika: {memory_used:.2f} MB")


### Rezultati poizvedbe  
Najbližji sosedi znotraj kategorije 'Musical Instruments' (10 rezultatov):  
ID: 5937, Title: Best Choice Products Beginner Acoustic Electric Guitar Starter Set 38in w/All Wood Cutaway Design, Case, Strap, Picks, Tuner - Blue, Category: Musical Instruments, Distance: 0.8286  
ID: 4770, Title: GLARRY Full Size Electric Guitar for Beginner with Amp and Accessories Pack Guitar Bag (Dark blue), Category: Musical Instruments, Distance: 0.8520  
ID: 3673, Title: No Amp Required! ToneWoodAmp SOLO Multi-Effect Processor for Acoustic-Electric Guitars (Right Handed), Category: Musical Instruments, Distance: 0.8564  
ID: 5501, Title: Amyove 39 Inch Electric Guitar, Full Size Guitar Kit, Electric Guitar Kits for Beginners with Bag, Digital Tuner, Capo, Strap, Cable, Picks (Red), Category: Musical Instruments, Distance: 0.8806  
ID: 9897, Title: Electric Acoustic Guitar, Full Size 41 Inch Acoustic Guitar Cutaway Bundle with Pickups for Beginners Adults Teens, Matte Black, by Vangoa, Category: Musical Instruments, Distance: 0.8915  
ID: 8808, Title: Asmuse Headless Electric Guitar, Full Size Beginner Electric Guitar Kit, HH Pickup Solid Body Electric Guitar, Guitar Starter Set with Gig Bag and Accessories, Category: Musical Instruments, Distance: 0.8980  
ID: 2934, Title: Alvarez RD8C Acoustic Guitar, Category: Musical Instruments, Distance: 0.9022  
ID: 2550, Title: JD.Moon Electric Guitar Body for Fender Stratocaster Strat DIY Red Walnut Wood Unfinished, Category: Musical Instruments, Distance: 0.9032  
ID: 2150, Title: Leo Jaymz Electric Guitar Body for TL Style - Guitar Accessory DIY Green Ash Guitar Body, Category: Musical Instruments, Distance: 0.9062  
ID: 1650, Title: LyxPro 39” SB Series Electric Guitar, Les Paul-Style Kit for Beginner, Intermediate & Pro Players Solid Body Guitar, Bonus 2-Pack of Picks, Mahogany Wood, Volume/Tone Controls, 3-Way Pickup - Sunburst, Category: Musical Instruments, Distance: 0.9124  
#### BREZ INDEKSA
Čas izvajanja: 0.1459 s  
Poraba pomnilnika: 0.07 MB  
#### Z INDEKSOM  
Čas izvajanja: 0.0557 s  
Poraba pomnilnika: 0.12 MB  

## Poizvedba 3: Najbolj oddaljeni produkti glede na embedding

Ta poizvedba uporablja stolpec `embedding` za iskanje produktov, ki so najbolj oddaljeni od določenega ciljnega vektorja. 


In [ ]:
import time
import psutil
import os

# Inicializacija spremljanja pomnilnika
process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss  # Pomnilnik na začetku v bajtih

# Ciljni embedding za iskanje
target_embedding = model.encode("Acoustic drum set for beginners").tolist()

# Preverjanje dimenzije embeddinga
if len(target_embedding) != 384:
    raise ValueError(f"Embedding mora imeti dimenzijo 384, a ima {len(target_embedding)}.")

# Začetek merjenja časa
start_time = time.time()

# SQL poizvedba za pridobitev najbolj oddaljenih produktov
cur.execute("""
    SELECT id, title, main_category, embedding <-> %s::vector AS distance
    FROM products
    ORDER BY distance DESC
    LIMIT 10;
""", (target_embedding,))

# Prikaz rezultatov
results = cur.fetchall()
end_time = time.time()

# Merjenje pomnilnika po izvedbi
end_memory = process.memory_info().rss  # Pomnilnik na koncu v bajtih

# Izpis rezultatov
print(f"Najbolj oddaljeni produkti (10 rezultatov):")
for row in results:
    print(f"ID: {row[0]}, Title: {row[1]}, Category: {row[2]}, Distance: {row[3]:.4f}")

# Izračun in izpis časa izvajanja
elapsed_time = end_time - start_time
print(f"Čas izvajanja: {elapsed_time:.4f} s")

# Izračun in izpis porabe pomnilnika
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Pretvorimo v MB
print(f"Poraba pomnilnika: {memory_used:.2f} MB")


### Rezultati poizvedbe
Najbolj oddaljeni produkti (10 rezultatov):  
ID: 3931, Title: for Motorola Babysense Vtech Baby Monitor 9.7 Feet Micro-USB Charger Power Cord Replacement Adapter Supply Compatible with Parent Unit MBP33S MBP36S MBP36XL MBP38S MBP41S MBP43S MBP843 MBP853 MBP854, Category: All Electronics, Distance: 1.4917  
ID: 577, Title: UbiGear Technology Home 18kw Power Saver Save Electricity Energy 35% Less Money Us, Category: Computers, Distance: 1.4909  
ID: 7566, Title: Belkin Conserve Insight Electric Monitor, Category: Office Products, Distance: 1.4772  
ID: 7059, Title: Barbary Fig Seed Oil 100% pure organic anti aging Super moisturizing oil (1 fl oz), Category: All Beauty, Distance: 1.4709  
ID: 5394, Title: hi, Category: Health & Personal Care, Distance: 1.4658  
ID: 2623, Title: UpBright New Global 12V AC DC Adapter Compatible with Portable Oxygen Enrichment Enriched Machine MAF-605A MAF605A Power Supply Cord Cable Battery Charger Mains PSU, Category: All Electronics, Distance: 1.4632  
ID: 9279, Title: Compatible with iPhone Cable, Category: Industrial & Scientific, Distance: 1.4568  
ID: 9710, Title: Shure RMCE-BT2 High-Resolution Bluetooth 5.0 Communication Cable (Renewed), Category: Industrial & Scientific, Distance: 1.4565  
ID: 7904, Title: Blizzard Lighting DMX50Q 50 Foot 3-Pin DMX Cable, Category: All Electronics, Distance: 1.4562  
ID: 526, Title: Lynn Electronics 070 Visual Phone Strobe Flasher (2), Category: Industrial & Scientific, Distance: 1.4530  
#### Brez indeks
Čas izvajanja: 0.0936 s  
Poraba pomnilnika: 0.08 MB  
#### Z indeksom
Čas izvajanja: 0.0736 s  
Poraba pomnilnika: 0.07 MB  

## Poizvedba 4: Iskanje produktov z embeddingi, ki so podobni ciljnemu, z večdimenzionalnimi filtri

V tej poizvedbi iščemo najbližje izdelke ciljnemu embeddingu z dodatnimi pogoji, kot so prisotnost določenih ključnih besed v opisu in omejitve glede kategorij.

In [ ]:
import time
import psutil
import os

# Inicializacija spremljanja pomnilnika
process = psutil.Process(os.getpid())
start_memory = process.memory_info().rss  # Pomnilnik na začetku v bajtih

# Ciljni embedding za iskanje
target_embedding = model.encode("High-quality drum kits with advanced features").tolist()

# Preverjanje dimenzije embeddinga
if len(target_embedding) != 384:
    raise ValueError(f"Embedding mora imeti dimenzijo 384, a ima {len(target_embedding)}.")

# Začetek merjenja časa
start_time = time.time()

# SQL poizvedba za pridobitev najbližjih produktov z večdimenzionalnimi filtri
cur.execute("""
    SELECT id, title, main_category, description, embedding <-> %s::vector AS distance
    FROM products
    WHERE 'Drums & Percussion' = ANY(categories)  -- Pogoj za filtriranje po kategoriji
      AND description IS NOT NULL                 -- Preverimo, da opis ni NULL
      AND description ILIKE '%%advanced%%'        -- Pogoj za prisotnost besede 'advanced' v opisu
    ORDER BY distance ASC
    LIMIT 10;
""", (target_embedding,))

# Prikaz rezultatov
results = cur.fetchall()
end_time = time.time()

# Merjenje pomnilnika po izvedbi
end_memory = process.memory_info().rss  # Pomnilnik na koncu v bajtih

# Izpis rezultatov
print(f"Najbližji produkti z dodatnimi filtri (10 rezultatov):")
for row in results:
    print(f"ID: {row[0]}, Title: {row[1]}, Category: {row[2]}, Description: {row[3][:50]}..., Distance: {row[4]:.4f}")

# Izračun in izpis časa izvajanja
elapsed_time = end_time - start_time
print(f"Čas izvajanja: {elapsed_time:.4f} s")

# Izračun in izpis porabe pomnilnika
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Pretvorimo v MB
print(f"Poraba pomnilnika: {memory_used:.2f} MB")


### Rezultati poizvedbe
Najbližji produkti z dodatnimi filtri (10 rezultatov):  
ID: 4868, Title: Yamaha DTX502 Electronic Drum Trigger Module, Category: Musical Instruments, Description: {"With almost twice the wave ROM and more than 250..., Distance: 1.0459  
ID: 4710, Title: Eastar Cabasa Musical Instrument Percussion Instrument Hand Shaker with Stainless Steel Beads and Wooden Handle for Students Kids Classroom Band, Small Size, Category: Musical Instruments, Description: {"Eastar Would Never Disappoint You. Eastar Cabasa..., Distance: 1.0797  
ID: 4906, Title: Pearl MBS-3000 Marching Bass Drum Stand, Category: Musical Instruments, Description: {"This completely height-adjustable stand features..., Distance: 1.1491  
ID: 7570, Title: Meinl Cymbals GX-8/10ES Generation-X 8" and 10" Electro Stack Cymbal Pair (VIDEO), Category: Musical Instruments, Description: {"The GENERATION X ELECTRO STACKS have a hard, siz..., Distance: 1.1964  
ID: 1179, Title: SENSOR ACTUATOR Fits ROLAND FD-8, TD-1 TD-11 TD-15 TD-17 Hi Hat Pedal Rubber Part (One Rubber), Category: Musical Instruments, Description: {"Name: Hi Hat Pedal SENSOR and Rubber part Suit f..., Distance: 1.1999  
#### Brez indeksa
Čas izvajanja: 0.2685 s  
Poraba pomnilnika: 0.05 MB  
#### Z indeksom
Čas izvajanja: 0.2140 s  
Poraba pomnilnika: 0.03 MB

## Uporaba indeksov na stolpcu embedding

Indeks na stolpcu `embedding` omogoča hitrejše izvajanje poizvedb, ki uporabljajo operator **`<->`** za izračun razdalje med vektorji. PostgreSQL omogoča uporabo indeksa vrste `ivfflat`, kar omogoča optimizacijo iskanja najbližjih sosedov v veliki množici podatkov.

### Koraki:
1. Ustvarimo indeks na stolpcu `embedding`.
2. Primerjamo čas izvajanja poizvedb **brez indeksa** in **z indeksom**.
3. Izvajamo enake poizvedbe kot prej in ocenimo izboljšave zmogljivosti.


In [42]:
# Ustvarimo indeks na stolpcu embedding
start_time = time.time()

cur.execute("""
    CREATE INDEX IF NOT EXISTS products_embedding_idx
    ON products
    USING ivfflat (embedding)
    WITH (lists = 100);  -- Parameter za zmogljivost indeksa
""")
conn.commit()

end_time = time.time()
print(f"Indeks uspešno ustvarjen. Čas: {end_time - start_time:.4f} s")


Indeks uspešno ustvarjen. Čas: 1.3894 s


## Zaključek

V tej nalogi smo raziskali uporabo PostgreSQL **pgvector** razširitve za delo z vektorskimi podatki in izvedli številne operacije, da smo ocenili zmogljivost ter uporabnost te tehnologije.

### Povzetek dela:
1. **Ustvarjanje in populacija baze podatkov:**
   - Ustvarili smo tabelo `products`, ki vključuje atribute, kot so `main_category`, `title`, `description`, `features`, `categories`, in **`embedding`** za vektorske predstavitve podatkov.
   - Generirali smo embeddinge za vse zapise z uporabo modela **SentenceTransformer** ter jih shranili v tabelo.

2. **CRUD operacije:**
   - Izvedli smo osnovne CRUD operacije (Create, Read, Update, Delete) na tabeli `products`, pri čemer smo merili čas izvajanja in porabo pomnilnika za vsako operacijo.

3. **Poizvedbe brez indeksa:**
   - Izvedli smo več poizvedb za iskanje najbližjih sosedov glede na embedding ciljne poizvedbe.
   - Implementirali smo različne scenarije, vključno s filtriranjem po kategorijah in ključnih besedah v opisu.
   - Merili smo čas izvajanja in porabo pomnilnika za vsako poizvedbo.

4. **Uporaba indeksa:**
   - Na stolpcu `embedding` smo ustvarili **ivfflat indeks**, ki omogoča optimizacijo iskanja najbližjih sosedov.
   - Ponovili smo ključne poizvedbe in opazovali izboljšave v hitrosti izvajanja ter učinkovitosti pomnilnika.

5. **Analiza rezultatov:**
   - Primerjali smo čas izvajanja in porabo pomnilnika za poizvedbe **brez indeksa** in **z indeksom**.
   - Ugotovili smo, da indeksiranje znatno izboljša zmogljivost, zlasti pri iskanju na večjih množicah podatkov.

### Ključne ugotovitve:
- **pgvector** je izjemno zmogljiva razširitev za delo z vektorskimi podatki znotraj PostgreSQL, kar omogoča učinkovito iskanje najbližjih sosedov.
- **Indeksiranje** bistveno izboljša zmogljivost pri velikih naborih podatkov in omogoča hitro iskanje relevantnih rezultatov.
- Integracija z modeli za generiranje embeddingov, kot je **SentenceTransformer**, omogoča preprosto ustvarjanje vektorskih predstavitev za kompleksne podatke.

### Zaključek:
S tem delom smo pokazali, kako lahko z uporabo PostgreSQL **pgvector** razširitve in naprednih modelov za generiranje embeddingov izboljšamo analizo podatkov ter učinkovitost poizvedb na velikih množicah podatkov. To odpira vrata za uporabo v različnih aplikacijah, kot so priporočilni sistemi, iskalni mehanizmi in analitični modeli.

